# Segmentation

In [6]:
import os
from modalities.test_mask_segment import  segment_brain_images, mask_brain_images
from modalities.report_segmentation import traverse_and_generate_report

def run_segmentation_and_masking(input_folder: str, tpm_path: str):
    # searching for .nii files within the subdirectories of input_folder
    subject_ids = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.nii'):
                subject_ids.append(os.path.join(root, file))
    
    print(subject_ids)  # check .nii files

    for input_file in subject_ids:
        subject_id = os.path.basename(input_file).split('.')[0]
        subject_folder = os.path.dirname(input_file)
        segmentation_folder = os.path.join(subject_folder, 'segmentation')
        os.makedirs(segmentation_folder, exist_ok=True)
        
        for iteration in range(1, 4):
            segmentation_output = segment_brain_images(input_file, tpm_path, iteration, subject_id, subject_folder)
            if not segmentation_output:
                print(f"Segmentation failed for {input_file} in iteration {iteration}")
                break
            mask_output = mask_brain_images(os.path.join(segmentation_folder, f'{iteration}_segment'), subject_folder, subject_id, iteration)
            input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'masked_{subject_id}.nii')
            
            # renaming the masked output to the original file name for the next iter
            next_input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'{subject_id}.nii') 
            os.rename(input_file, next_input_file)
            input_file = next_input_file
    
    # Generate the report after all subjects have been processed
    traverse_and_generate_report(input_folder, input_folder)

# Example usage:

# 'data/analysis_conditions/'
input_folder = '/home/sharapova/raw_analysis/new_minori/T2'
tpm_path = 'data/TPM'
run_segmentation_and_masking(input_folder, tpm_path)

['/home/sharapova/raw_analysis/new_minori/T2/Group/20240526_HF6_G2/T2_20240526_HF6_G2.nii', '/home/sharapova/raw_analysis/new_minori/T2/Group/20240526_HF6_G1/T2_20240526_HF6_G1.nii', '/home/sharapova/raw_analysis/new_minori/T2/Group/20240611_HF6_g4/T2_20240611_HF6_g4.nii', '/home/sharapova/raw_analysis/new_minori/T2/Group/20240612_HF6_g3/T2_20240612_HF6_g3.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240523_HF6_S2/T2_20240523_HF6_S2.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240522_HF6_S1/T2_20240522_HF6_S1.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240612_HF6_S4/T2_20240612_HF6_S4.nii', '/home/sharapova/raw_analysis/new_minori/T2/Single/20240524_HF6_S3/T2_20240524_HF6_S3.nii']
240719-11:17:05,657 nipype.workflow DEBUG:
	 (vbm_segment_1_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_1, vbm_segment_1_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_1): No edge data
240719-11:17:05,659 nipype.workflow DEBUG:
	 (vbm_segment_1_T2_20240526_HF6_G2.

stty: 'standard input': Inappropriate ioctl for device


240719-11:18:01,40 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_2, vbm_segment_2_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_2): No edge data
240719-11:18:01,41 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_2, vbm_segment_2_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_2): new edge data: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:18:01,43 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_2, vbm_segment_2_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_2): Edge data exists: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:18:01,44 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_2, vbm_segment_2_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_2): new edge data: {'connect': [('transformation_mat', 'transformation_mat'), ('native_class_images', 'native_class_images')]}
240719-11:18:01

stty: 'standard input': Inappropriate ioctl for device


240719-11:18:43,964 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_3, vbm_segment_3_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_3): No edge data
240719-11:18:43,965 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_3, vbm_segment_3_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_3): new edge data: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:18:43,966 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_3, vbm_segment_3_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_3): Edge data exists: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:18:43,966 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240526_HF6_G2.segment_T2_20240526_HF6_G2_3, vbm_segment_3_T2_20240526_HF6_G2.data_sink_T2_20240526_HF6_G2_3): new edge data: {'connect': [('transformation_mat', 'transformation_mat'), ('native_class_images', 'native_class_images')]}
240719-11:1

...

240719-11:33:37,488 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_2, vbm_segment_2_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_2): No edge data
240719-11:33:37,489 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_2, vbm_segment_2_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_2): new edge data: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:33:37,489 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_2, vbm_segment_2_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_2): Edge data exists: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:33:37,490 nipype.workflow DEBUG:
	 (vbm_segment_2_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_2, vbm_segment_2_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_2): new edge data: {'connect': [('transformation_mat', 'transformation_mat'), ('native_class_images', 'native_class_images')]}
240719-11:3

stty: 'standard input': Inappropriate ioctl for device


240719-11:34:23,81 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_3, vbm_segment_3_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_3): No edge data
240719-11:34:23,82 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_3, vbm_segment_3_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_3): new edge data: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:34:23,82 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_3, vbm_segment_3_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_3): Edge data exists: {'connect': [('transformation_mat', 'transformation_mat')]}
240719-11:34:23,83 nipype.workflow DEBUG:
	 (vbm_segment_3_T2_20240524_HF6_S3.segment_T2_20240524_HF6_S3_3, vbm_segment_3_T2_20240524_HF6_S3.data_sink_T2_20240524_HF6_S3_3): new edge data: {'connect': [('transformation_mat', 'transformation_mat'), ('native_class_images', 'native_class_images')]}
240719-11:34:23

stty: 'standard input': Inappropriate ioctl for device


FileNotFoundError: No such file or no access: '/home/sharapova/raw_analysis/new_minori/T2/Group/20240526_HF6_G2/segmentation/3_segment/native_class_images/c120240526_HF6_G2.nii'

# Dartel

In [18]:
%load_ext autoreload
%autoreload 2
import os
from modalities.dartel import create_dartel_structure, perform_dartel, copy_segment_files, find_subject_id

def main(base_path: str):
    condition_folders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
    dartel_dir = create_dartel_structure(base_path)
    copy_segment_files(base_path, condition_folders, dartel_dir)
    perform_dartel(dartel_dir)

main('/home/sharapova/vbm_pipline/data/analysis_conditions')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/native_class_images/c1T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/native_class_images/c2T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c2/c2T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/dartel_input_images/rc1T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/rc1/rc1T2_176.nii
Copied /home/sharapova/vbm_pipline/data/analysis_conditions/Treated_by_psilocin_analysis/T2_176/segmentation/3_segment/dartel_input_images/rc2T2_176.nii to /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/

stty: 'standard input': Inappropriate ioctl for device


# Normalization

In [7]:
%load_ext autoreload
%autoreload 2
import os
from modalities.normalization import normalize_brain_images

root_derivatives_dir_path = os.path.abspath('/home/sharapova/vbm_pipline/data/analysis_conditions')
output_files = normalize_brain_images(root_derivatives_dir_path)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c1_files: ['/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_174.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_176.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_177.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_178.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_201.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_223.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_224.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_225.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_226.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_227.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/c1/c1T2_228.nii']
flow_field_files: ['/home/sharap

# Smoothing

In [8]:
%load_ext autoreload
%autoreload 2
import os
from modalities.smoothing import smooth_brain_images

root_derivatives_dir_path = os.path.abspath('/home/sharapova/vbm_pipline/data/analysis_conditions')

# Define the full width at half maximum (FWHM) for smoothing
fwhm = [3, 3, 3] 


output_files = smooth_brain_images(root_derivatives_dir_path, fwhm)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
normalize_files: ['/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_174.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_176.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_177.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_178.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_201.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_223.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_224.nii', '/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/warped_files/warped_files/mwc1T2_225.nii', '/home

240722-15:10:52,343 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/work/vbm_smoothing/data_sink/_report/report.rst"
240722-15:10:52,347 nipype.workflow INFO:
	 [Node] Executing "data_sink" <nipype.interfaces.io.DataSink>
240722-15:10:52,349 nipype.workflow INFO:
	 [Node] Finished "data_sink", elapsed time 0.000666s.
240722-15:10:52,350 nipype.workflow DEBUG:
	 Needed files: /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_174.nii;/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_174.mat;/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_176.nii;/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/smwc1T2_176.mat;/home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/output/smoothed_files/smoothed_files/s

# Statistical modeling and estimation

In [9]:
%load_ext autoreload
%autoreload 2
import os
from modalities.statistical_modeling_t_test_model import create_two_sample_t_test_model

contrasts_name = 'example-contrast'
output_files = create_two_sample_t_test_model('/home/sharapova/vbm_pipline/data/analysis_conditions', contrasts_name)
print(output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Detected conditions: ['control_analysis', 'Treated_by_psilocin_analysis']
240722-15:11:09,532 nipype.workflow DEBUG:
	 (vbm_stats_modeling.vbm_stats_modeling, vbm_stats_modeling.data_sink): No edge data
240722-15:11:09,532 nipype.workflow DEBUG:
	 (vbm_stats_modeling.vbm_stats_modeling, vbm_stats_modeling.data_sink): new edge data: {'connect': [('spm_mat_file', 'example-contrast.@spm_mat_file')]}
240722-15:11:09,538 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=4, mem_gb=28.10, cwd=/home/sharapova/vbm_pipline)
240722-15:11:09,542 nipype.workflow DEBUG:
	 Creating flat graph for workflow: vbm_stats_modeling


240722-15:11:09,544 nipype.workflow DEBUG:
	 expanding workflow: vbm_stats_modeling
240722-15:11:09,545 nipype.workflow DEBUG:
	 processing node: vbm_stats_modeling.vbm_stats_modeling
240722-15:11:09,545 nipype.workflow DEBUG:
	 processing node: vbm_stats_modeling.data_sink
240722-15:11:09,546 nipype.workflow DEBUG:
	 finished expanding workflow: vbm_stats_modeling
240722-15:11:09,546 nipype.workflow INFO:
	 Workflow vbm_stats_modeling settings: ['check', 'execution', 'logging', 'monitoring']
240722-15:11:09,552 nipype.workflow DEBUG:
	 PE: expanding iterables
240722-15:11:09,553 nipype.workflow DEBUG:
	 Detected iterable nodes []
240722-15:11:09,553 nipype.workflow DEBUG:
	 PE: expanding iterables ... done
240722-15:11:09,555 nipype.workflow DEBUG:
	 Performing depth first search
240722-15:11:09,556 nipype.workflow INFO:
	 Running in parallel.
240722-15:11:09,570 nipype.workflow DEBUG:
	 Progress: 2 jobs, 0/0/1 (done/running/ready), 0/2 (pending_tasks/waiting).
240722-15:11:09,570 nip

# Contrast

In [10]:
%load_ext autoreload
%autoreload 2
import os
from modalities.statistical_analysis import perform_statistical_analysis

output_files = perform_statistical_analysis('/home/sharapova/vbm_pipline/data/analysis_conditions', 'control-treatment')
print("Output files:", output_files)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Contrasts name provided: control-treatment
Parsed contrast1_name: control
Parsed contrast2_name: treatment
Constructed contrast1: ('control>treatment', 'T', ['Group_{1}', 'Group_{2}'], [1, -1])
Constructed contrast2: ('control<treatment', 'T', ['Group_{1}', 'Group_{2}'], [-1, 1])
240722-15:11:20,573 nipype.workflow DEBUG:
	 (vbm_stats_analysis.estimate_contrast, vbm_stats_analysis.data_sink): No edge data
240722-15:11:20,575 nipype.workflow DEBUG:
	 (vbm_stats_analysis.estimate_contrast, vbm_stats_analysis.data_sink): new edge data: {'connect': [('spm_mat_file', 'control-treatment.@spm_mat'), ('spmT_images', 'control-treatment.@T'), ('con_images', 'control-treatment.@con')]}
240722-15:11:20,577 nipype.workflow DEBUG:
	 [MultiProc] Starting (n_procs=4, mem_gb=28.10, cwd=/home/sharapova/vbm_pipline)
240722-15:11:20,579 nipype.workflow DEBUG:
	 Creating flat graph for workflow: vbm_stats_analysis
24072

240722-15:11:20,707 nipype.workflow DEBUG:
	 Allocating vbm_stats_analysis.estimate_contrast ID=0 (0.20GB, 1 threads). Free: 27.90GB, 3 threads.
240722-15:11:20,709 nipype.workflow DEBUG:
	 [Node] Hashes: [('beta_images', [('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/beta_0001.nii', '205731dd2fb764d9fd09ece0befe8d90'), ('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/beta_0002.nii', '2becc22a0f9647bbc3d2546a5c621bc7')]), ('contrasts', [('control>treatment', 'T', ['Group_{1}', 'Group_{2}'], ['1.0000000000', '-1.0000000000']), ('control<treatment', 'T', ['Group_{1}', 'Group_{2}'], ['-1.0000000000', '1.0000000000'])]), ('group_contrast', True), ('mfile', True), ('residual_image', ('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/ResMS.nii', '23088ebdbd539312de00d8345b6dee30')), ('spm_mat_file', ('/home/sharapova/vbm_pipline/data/analysis_conditions/statistical_analysis/SPM.mat', '6e459c123e08d29f2ac8b947ff

stty: 'standard input': Inappropriate ioctl for device


240722-15:11:38,602 nipype.workflow INFO:
	 [Job 0] Completed (vbm_stats_analysis.estimate_contrast).
240722-15:11:38,604 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
240722-15:11:38,605 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.10/28.10, Free processors: 4/4.
240722-15:11:38,702 nipype.workflow DEBUG:
	 Allocating vbm_stats_analysis.data_sink ID=1 (0.20GB, 1 threads). Free: 27.90GB, 3 threads.
240722-15:11:38,704 nipype.workflow DEBUG:
	 [Node] Setting 3 connected inputs of node "data_sink" from 1 previous nodes.
240722-15:11:38,705 nipype.workflow DEBUG:
	 Resolving paths in outputs loaded from results file.
240722-15:11:38,706 nipype.workflow DEBUG:
	 output: spm_mat_file
240722-15:11:38,707 nipype.workflow DEBUG:
	 [Node] data_sink - setting input control-treatment.@spm_mat = /home/sharapova/vbm_pipline/data/analysis_conditions/Dartel/work/vbm_stats_analysis/estimate_contrast/SPM.mat
240722-15:11:38,707 nipype.wo